In [38]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold, cross_val_predict, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import f1_score

In [39]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [40]:
train = pd.read_csv('./data/train_oil.csv')
test  = pd.read_csv('./data/oil_test.csv')

In [41]:
train = train[train['Onshore/Offshore'] != 'ONSHORE-OFFSHORE']

In [42]:
len(train)

304

In [43]:
train.head(100)

,Field name,Reservoir unit,Country,Region,Basin name,Tectonic regime,Latitude,Longitude,Operator company,Onshore/Offshore,Hydrocarbon type,Reservoir status,Structural setting,Depth,Reservoir period,Lithology,Thickness (gross average ft),Thickness (net pay average ft),Porosity,Permeability
0,ZHIRNOV,MELEKESKIAN,RUSSIA,FORMER SOVIET UNION,VOLGA-URAL,COMPRESSION/EVAPORITE,51.0000,44.8042,NIZHNEVOLZHSKNET,ONSHORE,OIL,DECLINING PRODUCTION,FORELAND,1870,CARBONIFEROUS,SANDSTONE,262.0,33.0,24.0,30.00
1,LAGOA PARDA,LAGOA PARDA (URUCUTUCA),BRAZIL,LATIN AMERICA,ESPIRITO SANTO,EXTENSION,-19.6017,-39.8332,PETROBRAS,ONSHORE,OIL,NEARLY DEPLETED,PASSIVE MARGIN,4843,PALEOGENE,SANDSTONE,2133.0,72.0,23.0,350.00
2,ABQAIQ,ARAB D,SAUDI ARABIA,MIDDLE EAST,THE GULF,COMPRESSION/EVAPORITE,26.0800,49.8100,SAUDI ARAMCO,ONSHORE,OIL,REJUVENATING,FORELAND,6050,JURASSIC,LIMESTONE,250.0,184.0,21.0,410.00
3,MURCHISON,BRENT,UK /NORWAY,EUROPE,NORTH SEA NORTHERN,EXTENSION,61.3833,1.7500,CNR,OFFSHORE,OIL,NEARLY DEPLETED,RIFT,8988,JURASSIC,SANDSTONE,425.0,300.0,22.0,750.00
4,WEST PEMBINA,NISKU (PEMBINA L POOL),CANADA,NORTH AMERICA,WESTERN CANADA,COMPRESSION,53.2287,-115.8008,NUMEROUS,ONSHORE,OIL,UNKNOWN,FORELAND,9306,DEVONIAN,DOLOMITE,233.0,167.0,11.8,1407.00
5,UCHKYR,XV-1,UZBEKISTAN,FORMER SOVIET UNION,AMU DARYA,INVERSION/COMPRESSION/EXTENSION/EVAPORITE,40.1494,62.9906,SREDAZGAZPROM,ONSHORE,GAS,DECLINING PRODUCTION,INVERSION/RIFT,5443,JURASSIC,DOLOMITE,82.0,59.0,16.0,61.00
6,WESTHOPE SOUTH,CHARLES,USA,NORTH AMERICA,WILLISTON,COMPRESSION,48.8521,-101.0130,AMERADA HESS,ONSHORE,OIL,MATURE PRODUCTION,INTRACRATONIC,3275,CARBONIFEROUS,DOLOMITIC LIMESTONE,43.0,15.5,10.0,2.60
7,MCALLEN RANCH,LOWER VICKSBURG,USA,NORTH AMERICA,GULF OF MEXICO NORTHERN ONSHORE,GRAVITY/EXTENSION/SHALE/SYNSEDIMENTATION,26.6226,-98.3153,SHELL,ONSHORE,GAS,MATURE PRODUCTION,DELTA/PASSIVE MARGIN,9400,PALEOGENE,SANDSTONE,300.0,140.0,16.0,0.60
8,ELLIS RANCH,CLEVELAND,USA,NORTH AMERICA,ANADARKO,COMPRESSION/EROSION,36.2724,-100.7161,NUMEROUS,ONSHORE,GAS,DECLINING PRODUCTION,FORELAND,6500,CARBONIFEROUS,SHALY SANDSTONE,150.0,26.0,14.0,0.40
9,STRACHAN,LEDUC,CANADA,NORTH AMERICA,WESTERN CANADA,COMPRESSION,52.4500,-115.4286,BANF AND AQUITAINE,ONSHORE,GAS,NEARLY DEPLETED,FORELAND,12068,DEVONIAN,DOLOMITE,900.0,535.0,7.8,11.00


In [44]:
y = train['Onshore/Offshore'].copy()
y = y.map(lambda x: 1 if x == 'ONSHORE' else 0)
# y = y.map(lambda x: 1 if x == 'ONSHORE' else 0 if x == 'OFFSHORE' else 2 if x == 'ONSHORE-OFFSHORE' else x)

In [45]:
X = train.drop(columns=['Onshore/Offshore', 'Field name'] , errors='ignore')
X_test = test.drop(columns=['Field name'], errors='ignore')

In [46]:
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

In [47]:
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='none')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols)
    ], remainder='drop'
)

In [48]:
pipe = Pipeline([
    ('pre', preprocessor),
    ('clf', LogisticRegression())
])

In [49]:
param_grid = {
    'clf__C': [0.01, 0.1, 0.8, 0.9, 0.95, 1, 1.05, 1.1, 1.25, 1.5, 10],
    'clf__penalty': ['l2'],
    'clf__class_weight': [None, 'balanced']
}

In [50]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
gs = GridSearchCV(pipe, param_grid, scoring='f1', cv=cv, n_jobs=-1, verbose=1)
gs.fit(X, y)

Fitting 5 folds for each of 22 candidates, totalling 110 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('pre',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Latitude',
                                                                          'Longitude',
                                                                          'Depth',
                                                                          'Thickness '
                                                                          '(gross '
                                                                          'average '
                                                                          'ft)',
                                                                          'Thickness '
                                                                          '(net '
                                                                          'pay '
                                                                          'average '
                                                                          'ft)',
                                                                          'Porosity',...
                                                                          'Region',
                                                                          'Basin '
                                                                          'name',
                                                                          'Tectonic '
                                                                          'regime',
                                                                          'Operator '
                                                                          'company',
                                                                          'Hydrocarbon '
                                                                          'type',
                                                                          'Reservoir '
                                                                          'status',
                                                                          'Structural '
                                                                          'setting',
                                                                          'Reservoir '
                                                                          'period',
                                                                          'Lithology'])])),
                                       ('clf', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'clf__C': [0.01, 0.1, 0.8, 0.9, 0.95, 1, 1.05, 1.1,
                                    1.25, 1.5, 10],
                         'clf__class_weight': [None, 'balanced'],
                         'clf__penalty': ['l2']},
             scoring='f1', verbose=1)

In [51]:
print("Best params:", gs.best_params_)
print("Best CV f1:", gs.best_score_)

Best params: {'clf__C': 1.5, 'clf__class_weight': None, 'clf__penalty': 'l2'}
Best CV f1: 0.9348376070620763


In [52]:
oof_proba = cross_val_predict(gs.best_estimator_, X, y, cv=cv, method='predict_proba', n_jobs=-1)[:,1]
thresholds = np.linspace(0.1, 0.9, 1001)
best_thresh, best_f1 = 0.5, 0
for t in thresholds:
    f1 = f1_score(y, (oof_proba >= t).astype(int))
    if f1 > best_f1:
        best_f1, best_thresh = f1, t
print(f"Best threshold by OOF: {best_thresh:.3f} (f1={best_f1:.4f})")

Best threshold by OOF: 0.486 (f1=0.9351)


In [53]:
final_model = gs.best_estimator_
final_model.fit(X, y)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Latitude', 'Longitude',
                                                   'Depth',
                                                   'Thickness (gross average '
                                                   'ft)',
                                                   'Thickness (net pay average '
                                                   'ft)',
                                                   'Porosity',
                                                   'Permeability']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='none',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Reservoir unit', 'Country',
                                                   'Region', 'Basin name',
                                                   'Tectonic regime',
                                                   'Operator company',
                                                   'Hydrocarbon type',
                                                   'Reservoir status',
                                                   'Structural setting',
                                                   'Reservoir period',
                                                   'Lithology'])])),
                ('clf', LogisticRegression(C=1.5))])

In [54]:
proba_test = final_model.predict_proba(X_test)[:,1]
pred_test = (proba_test >= best_thresh).astype(int)

In [55]:
test_ids = test.index
submission = pd.DataFrame({
    'index': test_ids,
    'Onshore/Offshore': pred_test
})
submission.to_csv('submission_logreg.csv', index=False)